In [1]:
from models import LeNet5

In [ ]:
import torch
import random
import numpy as np
from scripts import MNISTCSV
from torch.utils.data import DataLoader

seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)


if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) 

batch_size = 64

train_csv_path = "data/MNIST/mnist_train.csv"
test_csv_path = "data/MNIST/mnist_test.csv"

train_dataset = MNISTCSV(train_csv_path)
test_dataset = MNISTCSV(test_csv_path)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
for batch in train_loader :
    print(batch[0].shape)
    break

torch.Size([64, 1, 28, 28])


In [4]:
for batch in train_loader:
    print(f"Min pixel value: {batch[0].min()}, Max pixel value: {batch[0].max()}")
    break

Min pixel value: 0.0, Max pixel value: 1.0


In [5]:
model = LeNet5()

In [ ]:
import torch.optim as optim
import itertools

optimizers = [optim.SGD]


lr_ranges = [
    [round(x/100, 100) for x in list(range(550,450, -1))],
     # 0.5, 0.45, ..., 0.15
]
lrs = list(itertools.chain(*lr_ranges))  


epochs = [1, 2]


hyperparameters = [
    {"optimizer": opt, "lr": lr, "epochs": ep}
    for ep in epochs
    for lr in lrs
    for opt in optimizers
]


print(f"Nombre total de combinaisons : {len(hyperparameters)}")
for i, hp in enumerate(hyperparameters[:15]):  
    print(f"Combinaison {i + 1} : {hp}")

Nombre total de combinaisons : 200
Combinaison 1 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.5, 'epochs': 1}
Combinaison 2 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.49, 'epochs': 1}
Combinaison 3 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.48, 'epochs': 1}
Combinaison 4 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.47, 'epochs': 1}
Combinaison 5 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.46, 'epochs': 1}
Combinaison 6 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.45, 'epochs': 1}
Combinaison 7 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.44, 'epochs': 1}
Combinaison 8 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.43, 'epochs': 1}
Combinaison 9 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.42, 'epochs': 1}
Combinaison 10 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.41, 'epochs': 1}
Combinaison 11 : {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.4, 'epochs': 1}
Combinaison 12 : {'

In [25]:

import torch.nn as nn
import torch.optim as optim
from scripts import train_model_and_save



for i, params in enumerate(hyperparameters):
    print(f"\n--- Training Model {i + 1} ---\n")
    print("params : ", params)

    
    model = LeNet5()

    
    optimizer = params["optimizer"](model.parameters(), lr=params["lr"])

    
    criterion = nn.CrossEntropyLoss()

    
    train_model_and_save(model, train_loader, criterion, optimizer, params["epochs"], learning_rate=params["lr"])


--- Training Model 1 ---

params :  {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.5, 'epochs': 1}
Epoch 1/1, Loss: 1.9265892987947728
Model saved to models\saved_models\model_f98a7d95-290a-4020-95dd-6d5f089d523c.pth
Metadata saved to models\saved_models_metadata\metadata_f98a7d95-290a-4020-95dd-6d5f089d523c.json

--- Training Model 2 ---

params :  {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.49, 'epochs': 1}
Epoch 1/1, Loss: 2.2155249328501445
Model saved to models\saved_models\model_ede26001-c15d-44b5-a58a-7d63e5fb0f97.pth
Metadata saved to models\saved_models_metadata\metadata_ede26001-c15d-44b5-a58a-7d63e5fb0f97.json

--- Training Model 3 ---

params :  {'optimizer': <class 'torch.optim.sgd.SGD'>, 'lr': 5.48, 'epochs': 1}
Epoch 1/1, Loss: 1.798635933063686
Model saved to models\saved_models\model_8a44e29b-4023-46eb-9d01-7c89f8a3c684.pth
Metadata saved to models\saved_models_metadata\metadata_8a44e29b-4023-46eb-9d01-7c89f8a3c684.json

--- Training Model 4 ---

params

In [4]:

import re 

def find_uuid(file_name :str ) -> str:
    pattern = r"model_([a-f0-9\-]+)\.pth"
    match = re.search(pattern, file_name)
    if match:
        extracted_string = match.group(1)  
        return extracted_string
    else:
        None

In [7]:
import os
from scripts import evaluate_model_v2, load_metadata, add_metadata, check_if_metadata_has_results
import torch
import json 

from scripts import load_model
from models import LeNet5

model_class = LeNet5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "models/saved_models"
folder_path_meta = "models/saved_models_metadata"

file_names = os.listdir(folder_path)

metadata_file_names = os.listdir(folder_path_meta)

print("Files in the folder:")
for file_name in file_names:
    model = load_model(os.path.join(folder_path, file_name), model_class)

    uiid = find_uuid(file_name)
    target_file = f"metadata_{uiid}.json"
    metadata_path = os.path.join(folder_path_meta, target_file)
    metadata = load_metadata(metadata_path)

    final_loss = metadata.get("final_loss", "N/A")
    learning_rate = metadata.get("learning_rate", "N/A")
    optimizer = metadata.get("optimizer", "N/A")
    epochs = metadata.get("epochs", "N/A")

    print(f"Model {uiid} :\n")
    print(f"Final Loss: {final_loss}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Optimizer: {optimizer}")
    print(f"Epochs: {epochs}")

    if check_if_metadata_has_results(metadata_path):
        pass 

    else :
        results = evaluate_model_v2(model, test_loader, device)
        metadata = add_metadata(metadata_path, results)

Files in the folder:
Model 02c4e111-5995-45df-8d6a-165cc07ed7d0 :

Final Loss: 2.313824599994017
Learning Rate: 4.66
Optimizer: SGD
Epochs: 2


c:\Users\cypri\OneDrive - CentraleSupelec\Bureau\Non_Linear_Systems\scripts\utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model

Accuracy: 0.10
Precision: 0.01
Recall: 0.10
F1-Score: 0.02
Model 0412b831-f1ed-4eca-a38c-46c4d88ef6b7 :

Final Loss: 2.102224628681313
Learning Rate: 5.36
Optimizer: SGD
Epochs: 2


c:\Users\cypri\anaconda3\envs\Torch_Intro_ML\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\cypri\OneDrive - CentraleSupelec\Bureau\Non_Linear_Systems\scripts\utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

Accuracy: 0.10
Precision: 0.01
Recall: 0.10
F1-Score: 0.02
Model 045437a3-b858-443d-bf9c-267762217d6d :

Final Loss: 1.8891467284768628
Learning Rate: 4.52
Optimizer: SGD
Epochs: 1


...

c:\Users\cypri\anaconda3\envs\Torch_Intro_ML\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\cypri\OneDrive - CentraleSupelec\Bureau\Non_Linear_Systems\scripts\utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode u

Accuracy: 0.20
Precision: 0.11
Recall: 0.20
F1-Score: 0.12
Model fe042a81-38d8-4a18-8e64-3cdb8b0a87fc :

Final Loss: 0.34006404092034964
Learning Rate: 4.72
Optimizer: SGD
Epochs: 1
Accuracy: 0.96
Precision: 0.96
Recall: 0.96
F1-Score: 0.96
Model fe3fdc73-81f7-4fb8-b1bb-84a8b713aa1d :

Final Loss: 2.1399576160063876
Learning Rate: 5.4
Optimizer: SGD
Epochs: 1


c:\Users\cypri\OneDrive - CentraleSupelec\Bureau\Non_Linear_Systems\scripts\utils.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model

Accuracy: 0.31
Precision: 0.19
Recall: 0.29
F1-Score: 0.21
Model febd0af1-a3f8-4b4f-a9a3-48586f32d7f5 :

Final Loss: 2.324412568037444
Learning Rate: 4.91
Optimizer: SGD
Epochs: 2
Accuracy: 0.11
Precision: 0.01
Recall: 0.10
F1-Score: 0.02


c:\Users\cypri\anaconda3\envs\Torch_Intro_ML\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import os
from scripts import (
    evaluate_model_v2,
    load_metadata,
    add_metadata,
    check_if_metadata_has_results,
)
import torch
import json

from scripts import load_model
from models import LeNet5

model_class = LeNet5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "models/saved_models"
folder_path_meta = "models/saved_models_metadata"

file_names = os.listdir(folder_path)

metadata_file_names = os.listdir(folder_path_meta)

print("Files in the folder:")
for file_name in file_names:
    model = load_model(os.path.join(folder_path, file_name), model_class)

    uiid = find_uuid(file_name)
    target_file = f"metadata_{uiid}.json"
    metadata_path = os.path.join(folder_path_meta, target_file)
    metadata = load_metadata(metadata_path)

    final_loss = metadata.get("final_loss", "N/A")
    learning_rate = metadata.get("learning_rate", "N/A")
    optimizer = metadata.get("optimizer", "N/A")
    epochs = metadata.get("epochs", "N/A")

    print(f"Model {uiid} :\n")
    print(f"Final Loss: {final_loss}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Optimizer: {optimizer}")
    print(f"Epochs: {epochs}")

    if check_if_metadata_has_results(metadata_path):
        pass

    else:
        results = evaluate_model_v2(model, test_loader, device)
        metadata = add_metadata(metadata_path, results)